In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

[nltk_data] Downloading package stopwords to /home/tomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
def ver_info_a_filtrar(df, col):
    cant_publicaciones_columna_no_nulo = df[~df[col].isnull()].shape[0]
    print('La cantidad de publicaciones con ' + col +' no nulo es:', cant_publicaciones_columna_no_nulo)
    print('La cantidad total de publicaciones es:', df.shape[0])

In [8]:
def generar_palabras_no_queridas(arr=None):
    '''Retorna un set con las palabras a excluir, arr, es un campo opcional para excluir tambien 
    las palabras en ese arreglo'''
    stop_words = set(stopwords.words('spanish'))
    puntuacion = [',', '.', '!', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '_', '\'', '\"', 
                  '(', ')', '[', ']', '{', '}', 'i', 'p', 'br', '<', '>', 'col.', '\\', '/', '$', 'm2', 
                  '..', '...', 'ii', '#', ';', ':', 'col']
    for x in puntuacion:
        stop_words.add(x)
    if arr:
        for x in arr:
            stop_words.add(x)
    return stop_words

In [11]:
def generar_palabras_mas_frecuentes(df, col, n):
    '''Esta funcion recibe un df, y genera la n palabras mas usadas la columna pasada por parametro, 
    retorna un arreglo ordenado de mas usadas a menos usadas'''
    reviews = df[col].str.cat(sep=' ')

    # Splitea los textos en palabras
    tokens = word_tokenize(reviews)

    # Fuera palabras no necesarias
    stop_words = generar_palabras_no_queridas()
    tokens = [w for w in tokens if not w in stop_words]

    vocabulary = set(tokens)
    
    # Calcula y ordena por frecuencia
    frequency_dist = nltk.FreqDist(tokens)
    mas_usadas = sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:n]
    return mas_usadas

In [6]:
def feature_cantidad_mas_frecuentes(df, col, n):
    '''Retorna el df pasado por parametro con la columna nueva "cant_palabras_mas_frecuentes", donde dice
    el numero de palabras mas frecuentes presentes en la columna pasada por parametro. n es la cantidad 
    palabras mas frecuentes a usar. Por ejemplo: `feature_cantidad_mas_frecuentes(df, 'titulo', 50)` genera 
    columna nueva `cant_palabras_mas_frecuentes_titulo`, donde se indica cuantas palabras frecuentes aparecen
    en el titulo, siendo las palabras mas frecuentes, la n palabras mas usadas entre todas las columnas 
    pasadas por parametro'''
    
    ver_info_a_filtrar(df, col)
    
    arr_mas_frecuentes = generar_palabras_mas_frecuentes(df, col, n)
    set_mas_frecuentes = set(arr_mas_frecuentes)
    
    def contador_mas_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_mas_frecuentes:
                contador = contador + 1
        return contador
    
    df['cant_palabras_mas_frecuentes_' + col] = df[~df[col].isnull()][col].apply(contador_mas_frecuentes)
    df['cant_palabras_mas_frecuentes_' + col] = df['cant_palabras_mas_frecuentes_' + col].fillna(0)
    
    return df

In [ ]:
def ejemplo():
    # Ejemplo de uso de las palabras mas frecuentes
    df_palabras = feature_cantidad_mas_frecuentes(df_train, 'titulo', 200)
    df_palabras[['id', 'titulo', 'cant_palabras_mas_frecuentes_titulo']].head()

La cantidad de publicaciones con titulo no nulo es: 234613
La cantidad total de publicaciones es: 240000


In [ ]:
def generar_palabras_menos_frecuentes(df, col, n):
    '''Esta funcion recibe un df, y genera la n palabras menos usadas la columna pasada por parametro, 
    retorna un arreglo ordenado de menos usadas a menos usadas'''
    reviews = df[col].str.cat(sep=' ')

    # Splitea los textos en palabras
    tokens = word_tokenize(reviews)

    # Fuera palabras no necesarias
    stop_words = generar_palabras_no_queridas()
    tokens = [w for w in tokens if not w in stop_words]

    vocabulary = set(tokens)
    
    # Calcula y ordena por frecuencia
    frequency_dist = nltk.FreqDist(tokens)
    menos_usadas = sorted(frequency_dist, key=frequency_dist.__getitem__, reverse=False)[0:n]
    return menos_usadas

In [ ]:
def feature_cantidad_menos_frecuentes(df, col, n):
    '''Retorna el df pasado por parametro con la columna nueva "cant_palabras_menos_frecuentes", donde dice
    el numero de palabras menos frecuentes presentes en la columna pasada por parametro. n es la cantidad 
    palabras menos frecuentes a usar. Por ejemplo: `cant_palabras_menos_frecuentes(df, 'titulo', 50)` genera 
    columna nueva `cant_palabras_menos_frecuentes_titulo`, donde se indica cuantas palabras menos frecuentes aparecen
    en el titulo, siendo las palabras menos frecuentes, la n palabras menos usadas entre todas las columnas 
    pasadas por parametro'''
    
    ver_info_a_filtrar(df, col)
    
    arr_menos_frecuentes = generar_palabras_menos_frecuentes(df, col, n)
    set_menos_frecuentes = set(arr_menos_frecuentes)
    
    def contador_menos_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_menos_frecuentes:
                contador = contador + 1
        return contador
    
    df['cant_palabras_menos_frecuentes_' + col] = df[~df[col].isnull()][col].apply(contador_menos_frecuentes)
    df['cant_palabras_menos_frecuentes_' + col] = df['cant_palabras_menos_frecuentes_' + col].fillna(0)
    
    return df

In [ ]:
def ejemplo():
    # Ejemplo de uso de las palabras menos frecuentes
    df_palabras = feature_cantidad_menos_frecuentes(df_train, 'titulo', 200)
    df_palabras[['id', 'titulo', 'cant_palabras_menos_frecuentes_titulo']].head()

In [63]:
def palabras_mas_usadas_en_mas_caros_baratos(df, col, n_palabras, n_caros_baratos, df_test=None, mirar_mas_caros=True):
    """Busca los n_caros mas caros del df, y verifica las n_palabras mas usadas."""
    """Para cada texto de la columna col, cuenta cuantas de estas palabras contiene"""
    """Si se pasa un df_test, hace la cuenta teniendo en cuenta las palabras obtenidas en df"""
    """Si mirar_mas_caros es True, va a buscar las palabras mas frecuentos de los mas caros, si fuera False,
    buscaria las palabras mas frecuentes de los mas baratos"""
    
    info = 'CAROS' if mirar_mas_caros else 'BARATOS'
    
    ver_info_a_filtrar(df, col)
    df_busqueda = df.nlargest(n_caros_baratos, 'precio') if mirar_mas_caros else df.nsmallest(n_caros_baratos, 'precio')
    arr_mas_frecuentes = generar_palabras_mas_frecuentes(df_busqueda, col, n_palabras)
    set_mas_frecuentes = set(arr_mas_frecuentes)

    def contador_mas_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_mas_frecuentes:
                contador = contador + 1
        return contador
    
    df['palabras_mas_frecuentes_' + info + '_' + col] = df[~df[col].isnull()][col].apply(contador_mas_frecuentes)
    df['palabras_mas_frecuentes_' + info + '_' + col] = df['palabras_mas_frecuentes_' + info + '_' + col].fillna(0)
    
    if df_test is None:
        return df, df_test
    
    df_test['palabras_mas_frecuentes_' + info + '_' + col] = df_test[~df_test[col].isnull()][col].apply(contador_mas_frecuentes)
    df_test['palabras_mas_frecuentes_' + info + '_' + col] = df_test['palabras_mas_frecuentes_' + info + '_' + col].fillna(0)
    
    return df, df_test

In [62]:
def ejemplo():
    df = df_train
    df_aut = df_test
    palabras_mas_usadas_en_mas_caros_baratos(df, 'titulo', 80, 200, df_aut, False)

La cantidad de publicaciones con titulo no nulo es: 234613
La cantidad total de publicaciones es: 240000


,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,palabras_mas_frecuentes_BARATOS_titulo
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0,1.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0,2.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0,5.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0,2.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.0,2.0,2.0,...,NaN,NaN,2015-02-08 00:00:00,0.0,0.0,0.0,0.0,0.0,650000.0,4.0
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.0,3.0,3.0,...,19.294665,-99.692916,2014-07-10 00:00:00,0.0,0.0,0.0,1.0,1.0,1940000.0,3.0
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.0,2.0,1.0,...,NaN,NaN,2015-03-03 00:00:00,0.0,0.0,0.0,0.0,0.0,3400000.0,2.0
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.0,4.0,0.0,...,19.366651,-99.082246,2014-12-26 00:00:00,1.0,0.0,0.0,1.0,1.0,2890000.0,1.0
